In [1]:
import csv
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from statistics import mean
from numpy.linalg import norm

## Load Human Prompts

In [45]:
human_prompt_files = [
    "../sentence_embeddings/data/amazonTurkDesPrompt1.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt2.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt3.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt4.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt5.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt6.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt7.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt8.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt9.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt10.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt11.csv",
    "../sentence_embeddings/data/amazonTurkDesPromptNA.csv",
]

human_prompts = []

for human_prompt in human_prompt_files:
    prompts = []
    with open(human_prompt, 'r') as file: #, encoding='Windows-1252') as file:
        csvreader = csv.reader(file)
        for row in csvreader:
            prompts.append(row[0])
    prompts = [i.replace('\n',' ') for i in prompts]
    human_prompts.append(prompts)

In [46]:
pd.DataFrame(human_prompts)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,ï»¿A jump rope that is also to be used as a ti...,A muscle stimulate band that can be worn over ...,"A simple, but strong band that has a hook to b...","A calf strengthener, similar to a finger stren...",A rubber ball that can be inflated / deflated ...,An x shaped resistance band that has 2 hand gr...,A mat that has electrosensitive pressure point...,A fold bike that could be used as a bike and a...,I think there should be a product that will he...,I am thinking of a portable exercise device.,...,A vest that makes you flex your chest muscles ...,A fold up tension devise that can fit into a s...,something very small that will allow you to wo...,A rowing type machine that can be folded up in...,This device is a dumbbell in which the weighte...,a small collapsible step that can be used to d...,How about an abdominal or muscle training MP3 ...,"Flexible, non-electric, ergonomic, compact","A collapsible, compact elliptical machine.",I am thinking of some sort of rubber band stro...
1,ï»¿A vest made of crystal quartz pieces connec...,Perhaps there could be something...a cord?...a...,a device which is attached to the joints so wh...,"When you blush, you put sensors on your cheeks...",People could wear kinetic bracelets and anklet...,solar equipment collects energy,A bicycle-powered television,have electrodes that can suction cup to our bo...,A battery that is charged by the energy from y...,A shoe that collects the energy generated when...,...,Im thinking of clothing that contians statice ...,I know skin conductance goes up when people ar...,A stationary bike with the wheel connected to ...,put pressure buttons in the bottoms of shoes a...,I am thinking of something simple like a watch...,sort of like roller blades that store the mech...,Energy from the motion of walking could be col...,A smart phone app that trackA smart phone app ...,"To collect energy from human motion, a station...",Power collecting shoes that joggers or top rac...
2,ï»¿with a candle that has marks on it,A clock or new redesigned clock that measures ...,waterfall clock tracking time by water ammount,Instead of relying on visual inputs these woul...,A small chart on your wrist,A giant hourglass,A clock that used binary to measure time.,Marking a piece of string with knots to measur...,A medication that lets you be so in tune with ...,a clock that acts more like a measure of the s...,...,"""Timeline"" is a flexible strip of LED lights e...",Digital self-drawn laser clock art. The design...,Use base 10 instead of base-6 numbers.,A block you put in the freezer and it turns bl...,We could divide time up into different color q...,The sun app. The sun app uses a satellite in ...,It would be the number of times my fan turned ...,A device that monitors the rotations and posit...,TIME IS INFINITE THEREFORE SOMETHING HAS TO BE...,light bulbs that dim as the sun goes down
3,"ï»¿""A tall cylindrical bottle","Like a party pop, this would be for celebratio...",a turn table which senses something underneath...,It would be a puffer kind of like a perfume bo...,Use the same container that weed killer comes ...,I thought of an automatic powdered sugar dispe...,Conveyor belt through a box that sprays the po...,A metal powder shaker with small circular hole...,An electrically powered sifter. Like a hand po...,A small long object that looks like a scanner....,...,A series of spinning rotors move several spray...,"This would be like a car wash, but instead of ...",a cannister that sprays and coats something of...,A small canister with a spigot at the top and ...,A container with a perforated lid to dispense ...,A small salt-shaker like item. This one has a ...,"A cup/tube cover, with multiple layers that ha...",I bake often at home and this prompt made me i...,A device that can be used while cooking and ba...,An automatic brush machine that could be adjus...
4,"ï»¿""A retractable gripper that has a central h...",A rotating shelf

In [4]:
human_prompts[0][2]

'A simple, but strong band that has a hook to be able to attach to any (thin) pole or sturdy object. The band will have a handle on one side to do pulling exercises for lats, curls for biceps, chest flys for pecs, arm extentions for triceps, etc. It will be versatile and lightweight, and can be used almost anywhere.'

## Load GPT Base Prompts

In [5]:
gpt_base_prompt_files = [
    "../sentence_embeddings/data/Design_Prompt1.csv",
    "../sentence_embeddings/data/Design_Prompt2.csv",
    "../sentence_embeddings/data/Design_Prompt3.csv",
    "../sentence_embeddings/data/Design_Prompt4.csv",
    "../sentence_embeddings/data/Design_Prompt5.csv",
    "../sentence_embeddings/data/Design_Prompt6.csv",
    "../sentence_embeddings/data/Design_Prompt7.csv",
    "../sentence_embeddings/data/Design_Prompt8.csv",
    "../sentence_embeddings/data/Design_Prompt9.csv",
    "../sentence_embeddings/data/Design_Prompt10.csv",
    "../sentence_embeddings/data/Design_Prompt11.csv",
    "../sentence_embeddings/data/Design_PromptNA.csv",
]

gpt_base_prompts = []

for gpt_base_prompt in gpt_base_prompt_files:
    prompt = pd.read_csv(gpt_base_prompt, sep=",", header=None)
    lst = []
    for d in prompt.values:
        lst.append(d[0])
    prompt = []
    for i in range(len(lst)):
        mod_statements = lst[i].replace(lst[i][:3], '')
        prompt.append(mod_statements)

    gpt_base_prompts.append(prompt)

In [6]:
pd.DataFrame(gpt_base_prompts)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,A foldable jump rope that takes up minimal sp...,A collapsible hand bike with self-powered feat...,A set of adjustable dumbbells with expandable ...,A rolling mat that can be used for yoga and pi...,A lightweight stationary bike with portable pe...,A portable yoga block to help improve balance ...,A resistance band set that easily fits in a su...,A mini trampoline with a sturdy frame,An adjustable weight vest with interchangeable...,A travel-size medicine ball with a handle for...,...,An exercise ball that can be compressed for t...,A foldable balance board for core strength tr...,"A light weight, portable resistance band with...",A set of adjustable ankle weights with a carr...,A lightweight spinning wheel with adjustable ...,A folding rowing machine with multiple intens...,A yoga balance board that can be easily store...,A travel-size Pilates machine with adjustable...,A mini trampoline with a folding frame,. A jump rope with rechargeable batteries for ...
1,Wristband with kinetic energy harvesting capab...,Handheld exercise ball with embedded kinetic e...,Ankle bracelet that harvests energy from your ...,Resistance band with built-in kinetic energy h...,Arm and leg bands with built-in kinetic energy...,Jump rope with embedded kinetic energy harvest...,Braided rope with built-in kinetic energy harv...,Balance board with embedded kinetic energy har...,Suspension trainer with built-in kinetic energ...,Exercise machine with built-in kinetic energy...,...,Core roller with built-in kinetic energy harv...,Power roller with embedded kinetic energy har...,Body sculpting bars with built-in kinetic ene...,balance steps with embedded kinetic energy har...,Ab crunch machine with built-in kinetic energ...,Push-up bars with embedded kinetic energy har...,Pilates power ring with built-in kinetic ener...,Resistance bar with embedded kinetic energy h...,Pilates resistance loop with built-in kinetic...,. Weighted gloves with embedded kinetic energy...
2,Digital clock with color-changing LEDs indicat...,Wall clock made of ornaments that move along a...,Digital clock with a revolving disc showing a ...,Video wall featuring an animation of the sun a...,Digital clock with a series of concentric circ...,Mechanical wall clock with an hourglass and a ...,Wristwatch that uses a spinning disc to show t...,Digital clock with a bar that moves up and dow...,Wall clock with a series of bars that move up ...,Digital sundial with an LCD display showing t...,...,Digital clock with a set of wheels that move ...,Wristwatch with a set of arms that move to in...,Wall clock with a set of levers that move aro...,Digital clock with a set of colored blocks th...,Sundial with a series of symbols that move ar...,Digital clock with a set of sliding bars that...,Wristwatch with a rotating disc that changes ...,Wall clock with a set of symbols that move ar...,Digital clock with a set of planets that rota...,. Sundial with a series of rotating symbols th...
3,A handheld electric dispenser with variable se...,An overhead dusting device with a pivoting arm...,A rotating platform with a fan to disperse the...,A mechanical arm with a nozzle at the end to r...,A wall-mounted bird-cage-style powder dispense...,A motorized roller that applies the powder to ...,A wall-mounted powder sprayer with adjustable ...,A nozzle mounted on a tripod with a motor allo...,A stationary platform with a convection tube r...,A hand-cranked fan with adjustable settings f...,...,A wall-mounted system with an adjustable pump...,A stationary platform with adjustable setting...,A motorized platform with a spinning wheel to...,A robotic arm with a nozzle attached for prec...,A hand-held device with a rotating nozzle for...,A mechanized arm with an adjustable hopper to...,An automated sprayer with adjustable settings...,A handheld device with a vibrating nozzle to ...,A motorized rotating wheel with adjustable se...,. A stationary platform with ad

In [7]:
gpt_base_prompts[0][0]

' A foldable jump rope that takes up minimal space when not in use'

## Load GPT Unique Prompts

In [8]:
gpt_unique_files = [
    "../prompt_engineering/data/zero_shot/unique_1.csv",
    "../prompt_engineering/data/zero_shot/unique_2.csv",
    "../prompt_engineering/data/zero_shot/unique_3.csv",
    "../prompt_engineering/data/zero_shot/unique_4.csv",
    "../prompt_engineering/data/zero_shot/unique_5.csv",
    "../prompt_engineering/data/zero_shot/unique_6.csv",
    "../prompt_engineering/data/zero_shot/unique_7.csv",
    "../prompt_engineering/data/zero_shot/unique_8.csv",
    "../prompt_engineering/data/zero_shot/unique_9.csv",
    "../prompt_engineering/data/zero_shot/unique_10.csv",
    "../prompt_engineering/data/zero_shot/unique_11.csv",
    "../prompt_engineering/data/zero_shot/unique_12.csv"
]

gpt_unique_prompts = []

for gpt_unique_prompt in gpt_unique_files:
    try:
        prompt = pd.read_csv(gpt_unique_prompt, sep=",")
    except UnicodeDecodeError:
        prompt = pd.read_csv(gpt_unique_prompt, sep=",", encoding='Windows-1252')
    lst = []
    for d in prompt.values:
        lst.append(d[0])
    prompt = []
    for i in range(len(lst)):
        if type(lst[i]) is not float:
            mod_statements = lst[i].replace(lst[i][:3], '')
            prompt.append(mod_statements)

    gpt_unique_prompts.append(prompt)

In [9]:
pd.DataFrame(gpt_unique_prompts)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,Folding resistance bands with a built-in carry...,Elastic cord-based pull-up bar,Mini exercise bike with adjustable resistance,Sit-up board with resistance bands attached,Portable yoga mat with built-in resistance bands,Resistance band-based jump rope,Dumbbells with adjustable weights and a carryi...,Resistance tubing straps for working out arms ...,Small balance board for stretching and yoga,Mini trampoline with adjustable tension,...,Mini stepper with adjustable settings and res...,Resistance band with door attachment and adju...,Resistance bands with ankle straps for workin...,Resistance bands with adjustable tension and ...,Portable Pilates reformer with adjustable set...,Foam mat with adjustable resistance settings,Resistance bands with adjustable length and h...,Resistance band handle with adjustable settin...,Adjustable stretch straps with resistance bands,. Portable Pilates reformer with adjustable se...
1,A handheld device with a built-in crank for ha...,An ankle bracelet with a spinning wheel that c...,A modified wristwatch with a motorized generator,A belt with tiny turbines that spins around th...,A set of wrist weights with small wind turbine...,A weighted vest with motors that capture energ...,Shoes with built-in vibration motors that acti...,Ankle straps with small propellers that spin w...,Elastic bands with embedded piezoelectric gene...,A spinning wheel with a generator at the center,...,A pair of weighted shoes with an integrated g...,A pulley system connected to two weighted bar...,A weighted vest with an attached generator th...,A handheld device with an attached generator ...,An adjustable belt with multiple turbines con...,A pair of ankle weights with small turbines a...,A stationary bike with hand grips that contai...,A pair of hand weights with embedded motors f...,A set of jump ropes with small turbines attac...,. A weighted rod with small turbines attached ...
2,A clock made from a series of carefully crafte...,A wall-mounted clock that is driven by a perpe...,A solar-powered clock with a clock face that c...,A timer powered by a water wheel,A device that uses the earth’s gravitational p...,A chronometer powered by electromagnets,A clock powered by the changing tides,A sundial composed of a series of interlocking...,A pocket watch that reads the time from the mo...,A loupe that can be used to read the time fro...,...,A clock powered by the changing tides,A clock powered by the kinetic energy generat...,A timepiece that reads the time from the posi...,A timepiece powered by a mechanical calculator,A clockface made from a series of crystals th...,A clock with a special disk that rotates arou...,A sundial with a series of concentric circles...,A pocket watch that reads the time from the p...,A wristwatch with a built-in altimeter that m...,. A clock that reads the time from the positio...
3,A handheld device with a fan-shaped nozzle and...,A robotic arm with adjustable settings for the...,A sprinkler-style device with an adjustable se...,A ceiling-mounted device that rotates and disp...,A hand-held device with a sieve-type head and ...,A motorized arm with a swiveling head and a ho...,A rotating drum-style device with adjustable s...,A hand-held device with a tray-like head and a...,A grapnel-style device with adjustable setting...,A wall-mounted device with a fan-shaped nozzl...,...,A wall-mounted device with a spinning rotor a...,A ceiling-mounted device with a rotary nozzle...,A tabletop device with a tray-like head and a...,A handheld device with a roller-style head an...,A robotic arm with a fan-shaped nozzle and a ...,A wall-mounted device with a rotary nozzle an...,A ceiling-mounted device with an adjustable h...,A tabletop device with a fan-shaped nozzle an...,A hand-held device with a sieve-style head an...,. A robotic arm with a swiveling head and a re...
4,Retractable robotic arm with book hook,Telescoping grabber with suction cup head,

In [10]:
gpt_unique_prompts[0]

['Folding resistance bands with a built-in carrying case',
 'Elastic cord-based pull-up bar',
 'Mini exercise bike with adjustable resistance',
 'Sit-up board with resistance bands attached',
 'Portable yoga mat with built-in resistance bands',
 'Resistance band-based jump rope',
 'Dumbbells with adjustable weights and a carrying handle',
 'Resistance tubing straps for working out arms and legs',
 'Small balance board for stretching and yoga',
 ' Mini trampoline with adjustable tension',
 ' Portable chin-up bar with attachments for chin-ups and dips',
 ' Elastic resistance cable for exercising arms and legs',
 ' Portable pull-up bar with straps for hanging kettlebells',
 ' Mini stepper with adjustable resistance',
 ' Portable jumping rope with adjustable tension',
 ' Elastic resistance bands with handles and clips for attaching to doors and other objects',
 ' Foam roller with resistance bands attached',
 ' Portable weight bench with adjustable settings',
 ' Medicine ball with adjustabl

## Load GPT Diverse Prompts

In [11]:
gpt_files = [
    "../prompt_engineering/data/zero_shot/diverse_1.csv",
    "../prompt_engineering/data/zero_shot/diverse_2.csv",
    "../prompt_engineering/data/zero_shot/diverse_3.csv",
    "../prompt_engineering/data/zero_shot/diverse_4.csv",
    "../prompt_engineering/data/zero_shot/diverse_5.csv",
    "../prompt_engineering/data/zero_shot/diverse_6.csv",
    "../prompt_engineering/data/zero_shot/diverse_7.csv",
    "../prompt_engineering/data/zero_shot/diverse_8.csv",
    "../prompt_engineering/data/zero_shot/diverse_9.csv",
    "../prompt_engineering/data/zero_shot/diverse_10.csv",
    "../prompt_engineering/data/zero_shot/diverse_11.csv",
    "../prompt_engineering/data/zero_shot/diverse_12.csv"
]

gpt_diverse_prompts = []

for gpt_prompt in gpt_files:
    try:
        prompt = pd.read_csv(gpt_prompt, sep=",")
    except UnicodeDecodeError:
        prompt = pd.read_csv(gpt_prompt, sep=",", encoding='Windows-1252')
    lst = []
    for d in prompt.values:
        lst.append(d[0])
    prompt = []
    for i in range(len(lst)):
        if type(lst[i]) is not float:
            mod_statements = lst[i].replace(lst[i][:3], '')
            prompt.append(mod_statements)

    gpt_diverse_prompts.append(prompt)

In [12]:
pd.DataFrame(gpt_diverse_prompts)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,A spring-loaded resistance band that clips ont...,An inflatable yoga ball for balancing workouts.,A folding resistance trainer for full body wor...,A collapsible set of lightweight dumbbells wit...,A foldable jump rope.,A hand grip exerciser for strengthening your a...,"A mini, adjustable exercise bike for cycling w...",A cardio monitor that clips onto your shirt fo...,A folding step machine for stair climbing.,A set of weighted ankle straps for strengthen...,...,A mini exercise bike for indoor workouts.,A folding Pilates reformer for core and funct...,A collapsible fitness ladder for agility and ...,A weighted yoga belt for improved stretching ...,A folding walking stick for stability and bal...,A set of adjustable resistance bands for arms...,A lightweight agility ladder for improved coo...,A portable stepper for cardio workouts.,A yoga board for improved balance and coordin...,. A collapsible trampoline for rebounding exer...
1,A belt with a small generator attached which c...,A pair of shoes with a motor connected to the ...,A pair of wrist bands with a turbine that capt...,A device that attaches to the user's ankle and...,An arm band with a miniature battery attached ...,A wireless device that is strapped onto the us...,A pair of gloves with a small generator attach...,A hand-held device that grabs energy from the ...,An exercise bike with a generator attachment t...,A wristband with a series of gears connected ...,...,A pair of ankle weights with a generator atta...,A headband with a small motor attached to it ...,An armband with a battery attached to it whic...,A stationary bike fitted with a turbine attac...,A pair of gloves with a mini turbine attached...,An ankle strap with a small motor attached to...,A vest with a turbine attached to it which ca...,A pair of pants with a generator connected to...,A wireless device that is strapped onto the u...,. A pair of shoes with a battery connected to ...
2,An app that measures time passed based on the ...,Smartwatch that reads a person’s heart rate an...,An interactive clock that turns a specific col...,An automated timer that can be set to measure ...,A wall clock that has a touch-sensitive screen...,A bracelet with built-in sensors that detect m...,A programmable timer that can be set to measur...,A digital timer where users can select the des...,A timer app with a progress bar that changes c...,A time tracker that records time elapsed over...,...,A time tracker that records time passed in va...,An IoT-enabled timer that records the time sp...,A timer with a digital display that records t...,An interactive clock that plays music when a ...,An app that records the time spent on a certa...,A timer that measures the time passed since a...,A timer with a voice-activated feature that r...,A timer with a graphical display that shows t...,An app with a timer and a progress bar that c...,. A timer with a digital display that shows th...
3,A handheld wand that releases a light puff of ...,A wall-mounted dispenser that can be adjusted ...,A battery-operated device with a spinning whee...,A bell-shaped dispenser that releases the powd...,A manual crank device that is used to spread t...,A device that uses a fan to disperse a light m...,A hand-held device with several nozzles that c...,A device with a large wheel and a cylinder of ...,A mechanical device that uses centrifugal forc...,A large helium-filled balloon that releases t...,...,A long-handled squeegee that can be used to s...,A hand-held device with an adjustable nozzle ...,A device with two cylinders that rotate to sp...,A device with a fan and a compartment that re...,A dome-shaped device with adjustable nozzles ...,A handheld wand with a fan that blows the pow...,A bell-shaped device that releases the powder...,A device with a spinning wheel and a containe...,A hand-held device with a spinning wheel that...,. A wall-mounted device with a spinning wheel ...
4,A mechaniz

In [13]:
gpt_diverse_prompts[0]

['A spring-loaded resistance band that clips onto the back of your seat.',
 'An inflatable yoga ball for balancing workouts.',
 'A folding resistance trainer for full body workouts.',
 'A collapsible set of lightweight dumbbells with adjustable weights.',
 'A foldable jump rope.',
 'A hand grip exerciser for strengthening your arms.',
 'A mini, adjustable exercise bike for cycling workouts.',
 'A cardio monitor that clips onto your shirt for tracking distance and speed.',
 'A folding step machine for stair climbing.',
 ' A set of weighted ankle straps for strengthening lower body muscles.',
 ' A strap-on hydration system for running and biking.',
 ' A travel set of lightweight push-up bars.',
 ' A hand-held crossfit machine for full body workouts.',
 ' An ab roller wheel for core training.',
 ' Resistance band loops for arms, legs, and core.',
 ' An adjustable yoga strap for stretching.',
 ' A flexible jump-rope that can be coiled up into a ball.',
 ' An inflatable weight vest for adde

## Load GPT Novel Prompts

In [14]:
gpt_files = [
    "../prompt_engineering/data/zero_shot/novel_1.csv",
    "../prompt_engineering/data/zero_shot/novel_2.csv",
    "../prompt_engineering/data/zero_shot/novel_3.csv",
    "../prompt_engineering/data/zero_shot/novel_4.csv",
    "../prompt_engineering/data/zero_shot/novel_5.csv",
    "../prompt_engineering/data/zero_shot/novel_6.csv",
    "../prompt_engineering/data/zero_shot/novel_7.csv",
    "../prompt_engineering/data/zero_shot/novel_8.csv",
    "../prompt_engineering/data/zero_shot/novel_9.csv",
    "../prompt_engineering/data/zero_shot/novel_10.csv",
    "../prompt_engineering/data/zero_shot/novel_11.csv",
    "../prompt_engineering/data/zero_shot/novel_12.csv"
]

gpt_novel_prompts = []

for gpt_prompt in gpt_files:
    try:
        prompt = pd.read_csv(gpt_prompt, sep=",")
    except UnicodeDecodeError:
        prompt = pd.read_csv(gpt_prompt, sep=",", encoding='Windows-1252')
    lst = []
    for d in prompt.values:
        lst.append(d[0])
    prompt = []
    for i in range(len(lst)):
        if type(lst[i]) is not float:
            mod_statements = lst[i].replace(lst[i][:3], '')
            prompt.append(mod_statements)

    gpt_novel_prompts.append(prompt)

In [15]:
pd.DataFrame(gpt_novel_prompts)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,A foldable balance board with a carrying handle.,A collapsible resistance band with a detachabl...,A compact skipping rope with a counter display.,A wrist band weights band that is easily adjus...,A stretchable exercise band with a detachable ...,A pocket-sized table tennis set with paddles a...,A portable rowing machine with a computerized ...,A foldable yoga mat with detachable blocks.,A stretching harness with adjustable straps.,A foldable elliptical machine with built-in r...,...,A compact treadmill with a tracker display.,An aerobic stepper with a foldable resin frame.,A pull-up bar with an adjustable height.,An abdominal training device with a foldable ...,A strap for stretching with adjustable settings.,A leg press machine with a foldable frame.,A resistance tube with adjustable intensity.,A foldable exercise ball with a carrying case.,A bodyweight exercise system with adjustable ...,. A stationary bike with a foldable design.
1,A set of exercise-focused shoes that use piezo...,An exercise bike with a generator attached to ...,An adjustable weighted vest that collects ener...,A hand-held device that turns kinetic energy i...,An elliptical machine outfitted with a motor a...,A jump rope with an adjustable speed control a...,A stationary bike that utilizes an internal ge...,An exercise mat fitted with piezoelectric mate...,A generator-powered rowing machine that uses t...,A treadmill fitted with a generator to conver...,...,None,None,None,None,None,None,None,None,None,None
2,A clock with a rotating sundial that adjusts t...,"A 3D clock that has several rotating discs, ea...",A clock with a series of rotating quartz cryst...,A clock with a series of colored arrows that p...,A clock with a digital display that shows the ...,A clock with a hand that moves counter clockwi...,A clock with a series of concentric rings each...,A clock with a series of rotating stars that r...,A clock with a series of constellations that r...,A clock with a series of blinking LED lights ...,...,A clock with a series of illuminated planets ...,A clock with a series of colored blocks that ...,A clock with a series of gears that move to c...,A clock with a series of rotating planets tha...,A clock with a series of illuminated lights t...,A clock with a series of spinning disks that ...,A clock with a series of rotating wheels that...,A clock with a series of cartoons that move t...,A clock with a series of illuminated stars th...,. A clock with a series of discs that move to ...
3,A manually operated wand with a handle and a b...,A handheld blower with adjustable nozzles that...,A powered manual handgun-style device with a t...,A wheeled device with a series of rotating arm...,A stationary device that uses compressed air t...,An aerosol-style canister with a spray nozzle ...,A handheld device with a battery operated vibr...,A self-propelled machine with bristled wheels ...,A hand-pumped device with a long hose attached...,A manual device that uses gravity to drop the...,...,A hand-pumped device with a long hose that ca...,A manually operated device with a spinning wh...,A robotically operated device with a rotating...,An aerosol-style canister with a spray nozzle...,A stationary device with a vacuum tube that q...,A machine with an adjustable nozzle that accu...,An automated device with a robotic arm that p...,A powered manual handgun-style device with an...,A machine with an adjustable nozzle that crea...,. An electric fan-type device with adjustable ...
4,Telescopic book grabber,Book claw,Retractable robotic arm,Book retrieval wire,Automated ladder,Extendable robotic arm,Boomerang book dispenser,Magnetic book retreiver,Automated reaching arm,Robotic book retrieval system,...,Remote-controlled book-grabbing system,Robotic telescopic retrieval arm,Robotic book-carrying platform,Robotic book-grabbing vacuum cleaner,Automated conveyor belt book retriever,Robotic arm with telescopic book retrieva

## Embed data

In [16]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [47]:
human_embeddings = [model.encode(prompt) for prompt in human_prompts]
gpt_base_embeddings = [model.encode(prompt) for prompt in gpt_base_prompts]
gpt_unique_embeddings = [model.encode(prompt) for prompt in gpt_unique_prompts]
gpt_novel_embeddings = [model.encode(prompt) for prompt in gpt_novel_prompts]
gpt_diverse_embeddings = [model.encode(prompt) for prompt in gpt_diverse_prompts]

In [18]:
len(human_embeddings)

12

In [48]:
for embedding in [gpt_base_embeddings, gpt_unique_embeddings, gpt_novel_embeddings, gpt_diverse_embeddings]:
    for prompt in embedding:
        print(len(prompt))

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
20
100
100
100
100
100
20
100
100
100
20
100
20
100
100
30
20
100
100
100
100
100
100
100
100
100
100
100
30



# Similarity Metrics
## Nearest Generated Sample
Using cosine distance
Focus on design space coverage

In [62]:
distances_all = []

for embedding in [gpt_base_embeddings, gpt_unique_embeddings, gpt_novel_embeddings, gpt_diverse_embeddings]:
    distances_model = []
    for count, generated_embedding in enumerate(embedding):
        distances = []
        for original_datapoint in human_embeddings[count]:
            max_distance = -1e9
            for generated_datapoint in generated_embedding:
                distance = np.dot(original_datapoint, generated_datapoint)/(norm(original_datapoint)*norm(generated_datapoint))
                if distance > max_distance:
                    max_distance = distance
            distances.append(max_distance)
        mean_distance = mean(distances)
        # print(f"Mean distance of nearest generated samples for prompt {count} to human: {mean_distance}")
        distances_model.append(mean_distance)
    distances_all.append(distances_model)


In [53]:
distances_pd = pd.DataFrame(distances_all, index=['base', 'unique', 'novel', 'diverse']).transpose()
distances_pd.style.background_gradient()

,base,unique,novel,diverse
0,0.585067,0.585067,0.585067,0.585067
1,0.547175,0.547175,0.547175,0.547175
2,0.503856,0.503856,0.503856,0.503856
3,0.643280,0.643280,0.643280,0.643280
4,0.626648,0.626648,0.626648,0.626648
5,0.653445,0.653445,0.653445,0.653445
6,0.574444,0.574444,0.574444,0.574444
7,0.528263,0.528263,0.528263,0.528263
8,0.628966,0.628966,0.628966,0.628966
9,0.613280,0.613280,0.613280,0.613280


In [51]:
distances_pd

,base,unique,novel,diverse
0,0.585067,0.517619,0.599553,0.596939
1,0.547175,0.607453,0.582830,0.630043
2,0.503856,0.556782,0.508827,0.564358
3,0.643280,0.582312,0.636292,0.612928
4,0.626648,0.567196,0.642636,0.688466
5,0.653445,0.655240,0.639801,0.667657
6,0.574444,0.607188,0.552770,0.548061
7,0.528263,0.588945,0.655492,0.680270
8,0.628966,0.644406,0.642573,0.645146
9,0.613280,0.709398,0.700902,0.700261


In [22]:
distances_pd.mean(axis=0)

base       0.582375
unique     0.584000
novel      0.608263
diverse    0.618201
dtype: float32

In [23]:
distances_pd.mean(axis=1)

0     0.574794
1     0.591875
2     0.533456
3     0.618703
4     0.631237
5     0.654036
6     0.570616
7     0.613243
8     0.640273
9     0.680960
10    0.493355
11    0.575971
dtype: float32

### Mean distance

In [24]:
distances_all = []

for embedding in [gpt_base_embeddings, gpt_unique_embeddings, gpt_novel_embeddings, gpt_diverse_embeddings]:
    distances_model = []
    for count, generated_embedding in enumerate(embedding):
        distances = []
        for original_datapoint in human_embeddings[count]:
            max_distance = -1e9
            for generated_datapoint in generated_embedding:
                distance = np.dot(original_datapoint, generated_datapoint) / (
                            norm(original_datapoint) * norm(generated_datapoint))
                distances.append(distance)
        mean_distance = mean(distances)
        # print(f"Mean distance of nearest generated samples for prompt {count} to human: {mean_distance}")
        distances_model.append(mean_distance)
    distances_all.append(distances_model)

distances_pd = pd.DataFrame(distances_all, index=['base', 'unique', 'novel', 'diverse']).transpose()
distances_pd.style.background_gradient()

,base,unique,novel,diverse
0,0.324602,0.274185,0.360371,0.356714
1,0.338651,0.369838,0.443146,0.468018
2,0.342129,0.335781,0.362164,0.381621
3,0.474682,0.338983,0.499668,0.460262
4,0.316340,0.278414,0.402755,0.587810
5,0.506041,0.469429,0.553283,0.498135
6,0.263842,0.391706,0.307716,0.272439
7,0.415321,0.484806,0.512829,0.544298
8,0.446304,0.498284,0.456247,0.470654
9,0.316521,0.514502,0.538626,0.521498


In [25]:
distances_pd.mean(axis=0)

base       0.357931
unique     0.364939
novel      0.434605
diverse    0.438375
dtype: float32

### Max distance with combinatorial selection

In [73]:
from itertools import combinations


distances_all = []
std_all = []

for embedding in [gpt_base_embeddings, gpt_unique_embeddings, gpt_novel_embeddings, gpt_diverse_embeddings]:
    distances_prompt = []
    std_prompt = []
    for count in range(len(embedding)):
        distances_row = []
        iterate = list(range(len(np.array(embedding[count]))))
        max_distance = -1e9
        for combo in combinations(iterate, 2):
            # dot product
            distance = np.dot(human_embeddings[count][combo[0],:], embedding[count][combo[1],:])/(norm(human_embeddings[count][combo[0],:])*norm(embedding[count][combo[1],:]))

            if distance > max_distance:
                max_distance = distance
            distances_row.append(max_distance)
        # distances_prompt.append(distances_row)
        # print(f"Average Cosine Distance between human and generated embeddings: {np.mean(distances_row)}")
        distances_prompt.append(np.mean(distances_row))
        std_prompt.append(np.std(distances_row))
    distances_all.append(distances_prompt)
    std_all.append(std_prompt)

In [74]:
distances_pd = pd.DataFrame(distances_all, index=['base', 'unique', 'novel', 'diverse']).transpose()
distances_pd.style.background_gradient()

,base,unique,novel,diverse
0,0.737221,0.728106,0.787831,0.772523
1,0.627234,0.754285,0.693840,0.804790
2,0.727203,0.759167,0.757869,0.746776
3,0.777565,0.716466,0.786489,0.738319
4,0.796006,0.708150,0.796256,0.842014
5,0.807182,0.804529,0.686086,0.783779
6,0.767732,0.781708,0.716082,0.697302
7,0.704262,0.716625,0.740587,0.869470
8,0.770319,0.780850,0.753955,0.816280
9,0.764971,0.833361,0.839784,0.820882


In [75]:
std_pd = pd.DataFrame(std_all, index=['base', 'unique', 'novel', 'diverse']).transpose()
std_pd.style.background_gradient()

,base,unique,novel,diverse
0,0.048266,0.040348,0.054648,0.058639
1,0.047080,0.038874,0.050100,0.071106
2,0.080451,0.070154,0.087255,0.071355
3,0.096636,0.072523,0.106998,0.080442
4,0.052110,0.054450,0.041299,0.030477
5,0.074093,0.038099,0.069913,0.059435
6,0.014033,0.036022,0.014336,0.059737
7,0.026705,0.034384,0.043783,0.046754
8,0.090074,0.095994,0.097581,0.111882
9,0.040115,0.011729,0.021700,0.019006


# Novelty/Diversity Metrics
## Convex hull hypervolume

In [28]:
from scipy.spatial import ConvexHull
from sklearn.decomposition import PCA
from tqdm import tqdm

In [29]:
len(human_embeddings[0])

100

In [30]:
pca = PCA(n_components=3)

volumes_all = []
for prompts in tqdm([human_embeddings, gpt_base_embeddings, gpt_unique_embeddings, gpt_novel_embeddings, gpt_diverse_embeddings]):
    volumes_prompts = []
    for prompt in tqdm(prompts, leave=False):
        embeddings = pca.fit_transform(prompt)
        convex_hull = ConvexHull(embeddings)
        volume_prompt = convex_hull.volume
        volumes_prompts.append(volume_prompt)
    volumes_all.append(volumes_prompts)

100%|██████████| 5/5 [00:01<00:00,  4.19it/s]  


In [31]:
convex_hull_volumes_df = pd.DataFrame(volumes_all, index=['human', 'base', 'unique', 'novel', 'diverse']).transpose()
convex_hull_volumes_df.style.background_gradient()

,human,base,unique,novel,diverse
0,0.249209,0.324143,0.332000,0.254748,0.347631
1,0.243629,0.197065,0.326193,0.150178,0.174642
2,0.321649,0.238124,0.274756,0.160631,0.200796
3,0.317053,0.193241,0.346561,0.148948,0.190490
4,0.338390,0.367969,0.341639,0.264245,0.089237
5,0.261036,0.164197,0.197922,0.074321,0.187192
6,0.330279,0.307938,0.206876,0.245315,0.348458
7,0.276590,0.120958,0.094294,0.147922,0.161171
8,0.327088,0.196485,0.151242,0.301654,0.155180
9,0.306567,0.399708,0.175189,0.104635,0.190654


In [39]:
convex_hull_volumes_df

,human,base,unique,novel,diverse
0,0.249209,0.324143,0.332000,0.254748,0.347631
1,0.243629,0.197065,0.326193,0.150178,0.174642
2,0.321649,0.238124,0.274756,0.160631,0.200796
3,0.317053,0.193241,0.346561,0.148948,0.190490
4,0.338390,0.367969,0.341639,0.264245,0.089237
5,0.261036,0.164197,0.197922,0.074321,0.187192
6,0.330279,0.307938,0.206876,0.245315,0.348458
7,0.276590,0.120958,0.094294,0.147922,0.161171
8,0.327088,0.196485,0.151242,0.301654,0.155180
9,0.306567,0.399708,0.175189,0.104635,0.190654


In [32]:
convex_hull_volumes_df.mean(axis=0)

human      0.294032
base       0.259198
unique     0.266242
novel      0.185539
diverse    0.216658
dtype: float64

In [33]:
convex_hull_volumes_df.mean(axis=1)

0     0.301546
1     0.218341
2     0.239191
3     0.239259
4     0.280296
5     0.176934
6     0.287773
7     0.160187
8     0.226330
9     0.235351
10    0.284878
11    0.281919
dtype: float64

In [76]:
convex_hull_volumes_df.std(axis=1)


0     0.046073
1     0.069426
2     0.062672
3     0.086916
4     0.113562
5     0.067658
6     0.059672
7     0.069968
8     0.082791
9     0.117035
10    0.060381
11    0.111368
dtype: float64